# Decision Tree

We'll start by evaluating some a decision tree model using AUC to get the best score. We'll also review the Confusion Matrix for the best model on the validation data as well as looking at the models precision and recall.

---

## Imports and import Data

In [36]:
import jupyter_black

jupyter_black.load()

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    roc_auc_score,
)

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

from sklearn import tree

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

### Import the data

In [37]:
df = pd.read_csv("./final_data.csv")

In [38]:
df.shape

(72063, 18)

In [39]:
# Some records are noy unique
df.drop_duplicates(inplace=True)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71866 entries, 0 to 72062
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   block                 71866 non-null  object 
 1   iucr                  71866 non-null  object 
 2   primary_type          71866 non-null  object 
 3   description           71866 non-null  object 
 4   location_description  71866 non-null  object 
 5   arrest                71866 non-null  bool   
 6   domestic              71866 non-null  bool   
 7   beat                  71866 non-null  int64  
 8   district              71866 non-null  int64  
 9   ward                  71866 non-null  int64  
 10  community_area        71866 non-null  int64  
 11  fbi_code              71866 non-null  object 
 12  latitude              71866 non-null  float64
 13  longitude             71866 non-null  float64
 14  hour                  71866 non-null  int64  
 15  day                

### Set Target

The target feature is `arrest` which is a boolean feature. For the medel we need to change this to `0` and `1` values.

We will also change the `domestic` feature in the same manner

In [41]:
df.arrest = df.arrest.astype(int)
df.domestic = df.domestic.astype(int)

### Identify Catergorical and Numeric Columns

In [42]:
categorical_columns = [
    "iucr",
    "primary_type",
    "description",
    "location_description",
    "fbi_code",
    "zip",
    "street",
]

numerical_columns = [
    "domestic",
    "beat",
    "district",
    "ward",
    "community_area",
    "latitude",
    "longitude",
    "hour",
    "day",
]

features = categorical_columns + numerical_columns

---

# Simple Decision Tree

### Split the Data

The data will be split as follows:

#### Training

80% of the data will be used for Training. During training K-Fold cross validation will be used.

One the best model parameters are identified the full training dataset will be used to train the final model.

#### Test

20% of the data will be help back for final testing of the model.

In [43]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

dict_train = df_train[features].to_dict(orient="records")
dict_test = df_test[features].to_dict(orient="records")

dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(dict_train)
X_test = dv.transform(dict_test)

y_train = df_train.arrest.values
y_test = df_test.arrest.values

In [44]:
dict_train[0]

{'iucr': '1477',
 'primary_type': 'WEAPONS VIOLATION',
 'description': 'RECKLESS FIREARM DISCHARGE',
 'location_description': 'PARK PROPERTY',
 'fbi_code': '15',
 'zip': '066XX',
 'street': 'N, WESTERN, AVE',
 'domestic': 0,
 'beat': 2412,
 'district': 24,
 'ward': 50,
 'community_area': 2,
 'latitude': 42.001822361,
 'longitude': -87.689987495,
 'hour': 19,
 'day': 3}

In [45]:
# Setup parameters with a wide spread

cv_wide = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=999)
cv_fine = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=999)

# We'll use grid search but to speed things up only do a single parameter at a time
param_dict = {
    "max_depth": range(3, 25, 3),
    # "min_samples_leaf": range(10, 51, 10),
    # "min_samples_split": range(10, 51, 10),
}

dt = DecisionTreeClassifier(random_state=999)

---

### ROC AUC

In [46]:
# Do a wide search using AUC
grid = GridSearchCV(
    estimator=dt,
    param_grid=param_dict,
    cv=cv_wide,
    verbose=1,
    scoring="roc_auc",
    n_jobs=-1,
)

grid.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=1, n_splits=5, random_state=999),
             estimator=DecisionTreeClassifier(random_state=999), n_jobs=-1,
             param_grid={'max_depth': range(3, 25, 3)}, scoring='roc_auc',
             verbose=1)

In [47]:
# What is the best parameter value
grid.best_params_

{'max_depth': 18}

In [48]:
# What is the best score this parameter achieved
grid.best_score_

0.8226784454221363

In [49]:
# Add min samples leaf to the grid search parameters
param_dict = {
    "max_depth": [18],
    "min_samples_leaf": range(80, 161, 10),
    # "min_samples_split": range(10, 51, 10),
}

In [50]:
# Do a wide search using AUC
grid = GridSearchCV(
    estimator=dt,
    param_grid=param_dict,
    cv=cv_wide,
    verbose=1,
    scoring="roc_auc",
    n_jobs=-1,
)

grid.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=1, n_splits=5, random_state=999),
             estimator=DecisionTreeClassifier(random_state=999), n_jobs=-1,
             param_grid={'max_depth': [18],
                         'min_samples_leaf': range(80, 161, 10)},
             scoring='roc_auc', verbose=1)

In [51]:
# What is the best parameter value
grid.best_params_

{'max_depth': 18, 'min_samples_leaf': 100}

In [52]:
# What is the best score this parameter achieved
grid.best_score_

0.8588294410398604

In [53]:
# Add min samples split to the grid search parameters
param_dict = {
    "max_depth": [18],
    "min_samples_leaf": [100],
    "min_samples_split": range(3, 31, 3),
}

In [54]:
# Do a wide search using AUC
grid = GridSearchCV(
    estimator=dt,
    param_grid=param_dict,
    cv=cv_wide,
    verbose=1,
    scoring="roc_auc",
    n_jobs=-1,
)

grid.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=1, n_splits=5, random_state=999),
             estimator=DecisionTreeClassifier(random_state=999), n_jobs=-1,
             param_grid={'max_depth': [18], 'min_samples_leaf': [100],
                         'min_samples_split': range(3, 31, 3)},
             scoring='roc_auc', verbose=1)

In [55]:
# What is the best parameter value
grid.best_params_

{'max_depth': 18, 'min_samples_leaf': 100, 'min_samples_split': 3}

In [56]:
# What is the best score this parameter achieved
grid.best_score_

0.8588294410398604

---

## Evaluate the Model

In [57]:
dt_auc = DecisionTreeClassifier(
    max_depth=18,
    min_samples_leaf=100,
    min_samples_split=3,
)
dt_auc.fit(X_train, y_train)

y_pred = dt_auc.predict(X_test)

In [58]:
print("Confusion Matrix Tree : \n", confusion_matrix(y_test, y_pred), "\n")
print("The precision for Tree is ", precision_score(y_test, y_pred))
print("The recall for Tree is ", recall_score(y_test, y_pred), "\n")

Confusion Matrix Tree : 
 [[11111   174]
 [ 1410  1679]] 

The precision for Tree is  0.9060982191041554
The recall for Tree is  0.5435415992230496 

